In [28]:
import pandas as pd
import sqlite3
import json
from ast import literal_eval
from analysis_tool import whoscored_data_engineering
from analysis_tool import whoscored_custom_events

In [2]:
data = open('Data/unprocess data/Liverpool03_15_23.json', encoding='utf-8')
data = json.load(data)


In [3]:
event_data = data['matchCentreData']


In [5]:
player_id = event_data['playerIdNameDictionary']


In [6]:
match_id = data['matchId']

In [7]:
home_data = data['matchCentreData']['home']

In [8]:
home_team_name =  event_data['home']['name']



In [9]:
away_team_name =  event_data['away']['name']


In [10]:
home_teamID = event_data['home']['teamId']


In [11]:
away_teamID = event_data['away']['teamId']


In [12]:
events = event_data['events']


In [13]:
events = pd.DataFrame(events)


In [14]:
events['match_id'] = match_id

In [15]:
match_string = home_team_name +  " "  + "-" + " " + away_team_name

In [16]:
events['match_string'] = match_string

In [18]:
# Create a dictionary mapping team IDs to team names
team_name = {
    home_teamID: home_team_name,
    away_teamID: away_team_name
}
events['team_name'] = events['teamId'].map(team_name)




In [19]:
# Create a dictionary mapping team IDs to team names
team_dict = {home_teamID: 'Home', away_teamID: 'Away'}

# Add a column to the event data to indicate whether the corresponding data is for the home team or away team
events['Venue'] = events['teamId'].map(team_dict)


In [20]:
# Load the event ID json file
with open('data/event_id.json') as f:
    event_id = {value: key for key, value in json.load(f).items()}

# Replace the values in the 'satisfiedEventsTypes' column with the corresponding event type labels
events['satisfiedEventsTypes'] = events['satisfiedEventsTypes'].apply(lambda x: [event_id.get(i) for i in x])

In [21]:
# Extract the outcomeType values using the apply() function and a lambda function
events['outcomeType'] = events['outcomeType'].apply(lambda x: x['displayName'])

In [22]:
events['type'] = events['type'].apply(lambda x: x['displayName'])


In [24]:
events['period'] = events['period'].apply(lambda x: x['displayName'])


TypeError: string indices must be integers

In [25]:
events['qualifiers'] = events['qualifiers'].apply(lambda x: [{item['type']['displayName']: item.get('value', True)} for item in x])


In [26]:
events = events.rename(columns={'type': 'event_type'})

In [27]:
events['period'] = events['period'].map({'FirstHalf': 1, 'SecondHalf': 2})


In [29]:
events = whoscored_data_engineering.cumulative_match_mins(events)

In [30]:
events = whoscored_custom_events.insert_ball_carries(events)

In [31]:
events = whoscored_custom_events.get_xthreat(events)


In [32]:
for i, row in events.iterrows():
    player_id_val = row['playerId']
    if not pd.isnull(player_id_val):
        player_name = player_id.get(str(int(player_id_val)), 'Unknown')
        events.at[i, 'playerName'] = player_name
    else:
        events.at[i, 'playerName'] = 'Unknown'

In [33]:
events

,eventId,minute,second,teamId,x,y,expandedMinute,period,type,outcomeType,...,event_type,cumulative_mins,index,id,match_string,team_name,Venue,xThreat,xThreat_gen,playerName
0,2.0,0.0,0.0,26,0.0,0.0,0.0,1.0,NaN,Successful,...,Start,0.000,0.0,2.520481e+09,Real Madrid - Liverpool,Liverpool,Away,NaN,NaN,Unknown
1,2.0,0.0,0.0,52,0.0,0.0,0.0,1.0,NaN,Successful,...,Start,0.000,1.0,2.520481e+09,Real Madrid - Liverpool,Real Madrid,Home,NaN,NaN,Unknown
2,3.0,0.0,0.0,26,50.0,50.0,0.0,1.0,NaN,Successful,...,Pass,0.000,2.0,2.520481e+09,Real Madrid - Liverpool,Liverpool,Away,-0.004436,0.000000,Mohamed Salah
3,3.5,0.0,1.5,26,30.6,41.0,0.0,1.0,"{'value': 99, 'displayName': 'Carry'}",Successful,...,Carry,0.025,NaN,NaN,NaN,NaN,NaN,0.000757,0.000757,Ibrahima Konaté
4,4.0,0.0,3.0,26,34.9,36.0,0.0,1.0,NaN,Unsuccessful,...,Pass,0.050,3.0,2.520481e+09,Real Madrid - Liverpool,Liverpool,Away,NaN,NaN,Ibrahima Konaté
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,11531.0,51.0,NaN,52,66.8,70.3,54.0,2.0,NaN,Unsuccessful,...,OffsideGiven,NaN,895.0,2.520521e+09,Real Madrid - Liverpool,Real Madrid,Home,NaN,NaN,Vinícius Júnior
1891,910.0,0.0,0.0,52,0.0,0.0,7.0,NaN,NaN,Successful,...,End,0.000,1607.0,2.520524e+09,Real Madrid - Liverpool,Real Madrid,Home,NaN,NaN,Unknown
1892,836.0,0.0,0.0,26,0.0,0.0,7.0,NaN,NaN,Successful,...,End,0.000,1608.0,2.520524e+09,Real Madrid - Liverpool,Liverpool,Away,NaN,NaN,Unknown
1893,1.0,0.0,0.0,52,0.0,0.0,0.0,NaN,NaN,Successful,...,FormationSet,0.000,1609.0,2.520457e+09,Real Madrid - Liverpool,Real Madrid,Home,NaN,NaN,Unknown


In [34]:
events.to_csv('Data/liverpool03_15_23.csv')